<a href="https://colab.research.google.com/github/matiangelico/IA/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***MONTAMOS DRIVE***

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
from tensorflow.keras.preprocessing import  image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(directory ="/content/drive/My Drive/DatosColab/flowers50/",
                                                 target_size=(224,224),
                                                 #batch_size=2048,
                                                 class_mode='categorical',                                                 
                                                 shuffle=False,
                                                 seed=15
                                                 )

print(train_generator.__len__())
x1=np.concatenate([train_generator.next()[0] for i in range(train_generator.__len__())])
y1=np.concatenate([train_generator.next()[1] for i in range(train_generator.__len__())])
labels = list(train_generator.class_indices.keys())  
print(x1.shape)
print(y1.shape)
print(labels)

shuffler = np. random. permutation(len(y1))
x1 = x1[shuffler]
y1 = y1[shuffler]

num_class = y1.shape[1]

Found 8191 images belonging to 16 classes.
256
(8191, 224, 224, 3)
(8191, 16)
['astilbe', 'bellflower', 'black_eyed_susan', 'calendula', 'california_poppy', 'carnation', 'common_daisy', 'coreopsis', 'daffodil', 'dandelion', 'iris', 'magnolia', 'rose', 'sunflower', 'tulip', 'water_lily']


In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x1, y1, random_state=1,test_size=0.1)


In [43]:
lb = [labels[np.argmax(x)] for x in y1[:10]]
for lb_enc, lb in zip(y1[:10],lb):
    print(lb_enc,"-->",lb)

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] --> bellflower
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] --> carnation
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] --> bellflower
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] --> astilbe
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] --> tulip
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] --> iris
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] --> coreopsis
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] --> bellflower
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] --> iris
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] --> sunflower


In [44]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import  RMSprop, Adam, Adamax

# MODEL BASE VGG16
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

# SUMAMOS CAPAS

x = base_model.output

x = GlobalAveragePooling2D(name="GAP2D")(x)
x = Dense(128,activation='relu', name="Dense1")(x)
x = Dropout(0.5, name="Drop1")(x)
x = Dense(128,activation='relu', name="Dense2")(x) 
x = Dropout(0.5, name="Drop2")(x)
x = Dense(128,activation='relu', name="Dense3")(x) 
x = Dropout(0.5, name="Drop3")(x)
# CAPA SOFTMAX CON CANTIDAD DE CLASES PARA CLASIFICACION
preds = Dense(num_class, activation='softmax', name="My_Softmax")(x) 

# ARMO EL MODELO COMPLETO
model = Model(inputs=base_model.input, outputs=preds)


# FREZO TODAS LAS CAPAS DEL MODELO BASE
for layer in model.layers[:19]:
    layer.trainable = False

# VEO EL RESUMEN DE LA RED
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:

# OPTIMIZADOR
# opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=decay_rate, amsgrad=False)
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.1, amsgrad=False)
#opt2 = Adamax( learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.02)

# COMPILO EL MODELO - USO CATEGORICAL CROSSENTROPY COMO LOSS FUNCTION
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy'])

history = model.fit(x1, y1, epochs = 40)    

#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
#history = model.fit(x_train, y_train, epochs=5, validation_split=0.2,callbacks=[callback], verbose=1)


Epoch 1/40
256/256 [==============================] - 4206s 16s/step - loss: 4.1853 - categorical_accuracy: 0.0763
Epoch 2/40
158/256 [=================>............] - ETA: 26:41 - loss: 2.8368 - categorical_accuracy: 0.0910

In [ ]:
def plotLoss(result, extra_title=""):
    loss = result.history['loss']
    val_loss = result.history['val_loss']
    epochs = range(1, len(loss) + 1)
    
    plt.figure(figsize=(7,7))
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss '+extra_title)
    plt.xlabel('Epochs')
    plt.ylabel('Mean Square Error')
    plt.legend()
    plt.show()

def plotAcc(result, extra_title=""):
    acc = result.history['categorical_accuracy']
    val_acc = result.history['val_categorical_accuracy']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(7,7))
    plt.plot(epochs, acc, 'y', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation acc '+extra_title)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.ylim([0,1])
    plt.legend()
    plt.show()


#acc = history.history['categorical_accuracy']    

#loss = history.history['loss']

#epoc = range(len(acc))

#plt.plot(epoc, acc, 'b', label='Training categorical_accuracy')

#plt.title('Training accuracy')
#plt.legend()

#plt.figure()

#plt.plot(epoc, loss, 'b', label='Training loss')

#plt.title('Training  loss')
#plt.legend()

#plt.show()

In [ ]:
plotLoss(history, "Modelo 1 - Optimizador 1")

In [ ]:
plotAcc(history, "Modelo 1 - Optimizador 1")


In [ ]:
# prediction1 = model.predict(x1[0], verbose=1)
evaluation1 = model.evaluate(x1, y1)

In [ ]:
from numpy.ma.core import true_divide
from keras.applications.vgg16 import decode_predictions
import os

def clasifica(lista, path):
    file = open("/content/drive/My Drive/DatosColab/resultado50-60.txt","w")
    aciertos = 0
    fallas = 0
    for my_file in lista:
        image1 = load_img(path+my_file, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image1)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)

        # predict the probability across all output classes
        yhat = model.predict(image)
        pred_label = np.argmax(yhat)
        pred_label_cat = labels[pred_label]


        print("--------------------------------------------------")
        print("Archivo --> ",my_file)
        print("pred_label_cat -->",pred_label_cat)
        print("--------------------------------------------------")
        if (my_file[0]==pred_label_cat[0] and my_file[1]== pred_label_cat[1] and my_file[3]== pred_label_cat[3]):
            acerto = "true"
            aciertos +=1 
        else:
            acerto = "false"
            fallas +=1
        file.write("Archivo --> "+my_file+"  Prediccion --> "+pred_label_cat+" Correcto --> "+acerto+os.linesep)
        # # convert the probabilities to class labels
        # #label = decode_predictions(yhat)
        # # retrieve the most likely result, e.g. highest probability
        # label = label[0][0]
        # # print the classification
        # print('%s (%.2f%%)' % (label[1], label[2]*100))

        # import numpy as np

        #image1 = np.array(image1)
        #plt.figure()
        #plt.imshow(image1)
        #plt.title(pred_label_cat)
    file.write("Total aciertos --> "+str(aciertos)+" Total fallas --> "+str(fallas))
    file.close()

In [ ]:
import os 

pathTest = '/content/drive/My Drive/DatosColab/flowersTest/'
tests = os.listdir('/content/drive/My Drive/DatosColab/flowersTest/')

clasifica(tests, pathTest)